In [1]:
from bert_serving.client import BertClient
import numpy as np
import pandas as pd

In [2]:
from tqdm import tqdm
from preprocess import read_articles_from_file_list, read_predictions_from_file, label2index
bc = BertClient(ip='10.2.1.51')

Using TensorFlow backend.


In [3]:
a = bc.encode(['First do it', 'then do it right', 'then do it better'])

In [4]:
train_folder = "datasets/train-articles"  # check that the path to the datasets folder is correct,
dev_folder = "datasets/dev-articles"  # if not adjust these variables accordingly
train_labels_file = "datasets/train-task2-TC.labels"
dev_template_labels_file = "datasets/dev-task-TC-template.out"
task_TC_output_file = "baseline-output-TC.txt"

In [5]:
def clean_text(text):
    # text = text.lower()
    # text = text.replace('\'', '')
    text = text.replace('‘', ' \' ')
    text = text.replace('’', ' \' ')
    text = text.replace('“', ' \" ')
    text = text.replace('”', ' \" ')
    text = text.replace('â', ' \' ')

    text = text.replace('"', ' " ')
    text = text.replace('\'', ' \' ')

    text = text.replace('—', ' - ')
    text = text.replace('–', ' - ')
    text = text.replace('…', '...')
    text = text.replace('  ', ' ')
    text = text.strip()
    return text

In [6]:
articles = read_articles_from_file_list(train_folder)
dev_articles = read_articles_from_file_list(dev_folder)
ref_articles_id, ref_span_starts, ref_span_ends, train_gold_labels = read_predictions_from_file(train_labels_file)
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)
print("Loaded %d annotations from %d articles" % (len(ref_span_starts), len(set(ref_articles_id))))


Loaded 6129 annotations from 357 articles


In [7]:
def compute_features(articles, ref_articles_id, span_starts, span_ends):
    # only one feature, the length of the span
    print(type(span_starts), len(span_starts))
    print(type(span_ends), len(span_ends))
    data = []
    article_spans = []
    for i, ref_id in tqdm(enumerate(ref_articles_id)):
        # print(articles[ref_id], span_starts[i], span_ends[i])
        article = articles[ref_id]
        article_span = clean_text(article[int(span_starts[i]):int(span_ends[i])])
        data.append([article_span])
        article_spans.append(article_span)
        
    return article_spans

In [8]:
articles = compute_features(articles, ref_articles_id, ref_span_starts, ref_span_ends)
dev_articles = compute_features(dev_articles, dev_article_ids, dev_span_starts, dev_span_ends)

6129it [00:00, 321448.62it/s]
1063it [00:00, 275951.30it/s]

<class 'list'> 6129
<class 'list'> 6129
<class 'list'> 1063
<class 'list'> 1063


In [9]:
from keras.utils import to_categorical
from pprint import pprint
import pickle
from preprocess import label2index, index2label

In [10]:
GET_NEW_BERT_EMB = False

In [11]:
data_path = './bert_processed_data/'
if GET_NEW_BERT_EMB:
    articles_emb = bc.encode(articles)
    dev_articles_emb = bc.encode(dev_articles)
    pprint(set(train_gold_labels))

    labels = [label2index[x] for x in train_gold_labels]
    labels = to_categorical(np.asarray(labels))

    # save train data
    pickle.dump(articles_emb, open(data_path + 'train_x.p', 'wb'))
    pickle.dump(labels, open(data_path + 'train_y.p', 'wb'))

    # save dev data
    pickle.dump(dev_articles_emb, open(data_path + 'dev_x.p', 'wb'))
    
else:
    articles_emb = pickle.load(open(data_path + 'train_x.p', 'rb'))
    labels = pickle.load(open(data_path + 'train_y.p', 'rb'))
    dev_articles_emb = pickle.load(open(data_path + 'dev_x.p', 'rb'))

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_x, test_x, train_y, test_y = train_test_split(articles_emb,
                                                  labels,
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  stratify=labels)

In [14]:
from keras.layers import Conv2D, MaxPool2D, Bidirectional
from keras.layers import Embedding, Concatenate
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

def model_MLP(emb_size, out_size):
    model = Sequential()
    model.add(Dense(128, input_dim=emb_size, activation='relu'))
    model.add(Dropout(0.5))
    # model.add(Dense(64, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(out_size, activation='softmax'))
    print(model.summary())
    return model

In [15]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD
from models import load_model
dev_template_labels_file = "datasets/dev-task-TC-template.out"


emb_size = 1024
out_size = 14

model = model_MLP(emb_size, out_size)

lr = 0.0001
bz = 256
epochs = 300

opt = Adam(lr=lr)
# opt = SGD(0.01)
# print(str(opt))
# exit()
model_name = 'bert_text_Adam_lr%s_bz%s' % (lr, bz)
model_path = 'models/%s' % (model_name)
checkpoint = ModelCheckpoint('%s.{epoch:02d}.hdf5' % (model_path), monitor='loss', verbose=1,
                             save_best_only=False, mode='auto')

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])
try:
    model.fit(train_x, train_y, validation_data=[test_x, test_y], epochs=epochs,
              batch_size=bz,
              shuffle=True, callbacks=[checkpoint])
except:
    pass






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                1806      
Total params: 133,006
Trainable params: 133,006
Non-trainable params: 0
_________________________________________________________________
None


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4903 samples, validate on 1226 samples
Epoch 1/300
4903/4903 [==============================] - 1s 179us/step - loss: 3.0061 - acc: 0.0859 - val_loss

4903/4903 [==============================] - 0s 36us/step - loss: 1.3868 - acc: 0.5619 - val_loss: 1.3585 - val_acc: 0.5759

Epoch 00027: saving model to models/bert_text_Adam_lr0.0001_bz256.27.hdf5
Epoch 28/300
4903/4903 [==============================] - 0s 29us/step - loss: 1.3725 - acc: 0.5717 - val_loss: 1.3511 - val_acc: 0.5783

Epoch 00028: saving model to models/bert_text_Adam_lr0.0001_bz256.28.hdf5
Epoch 29/300
4903/4903 [==============================] - 0s 56us/step - loss: 1.3622 - acc: 0.5733 - val_loss: 1.3420 - val_acc: 0.5832

Epoch 00029: saving model to models/bert_text_Adam_lr0.0001_bz256.29.hdf5
Epoch 30/300
4903/4903 [==============================] - 0s 56us/step - loss: 1.3547 - acc: 0.5745 - val_loss: 1.3348 - val_acc: 0.5840

Epoch 00030: saving model to models/bert_text_Adam_lr0.0001_bz256.30.hdf5
Epoch 31/300
4903/4903 [==============================] - 0s 44us/step - loss: 1.3423 - acc: 0.5801 - val_loss: 1.3272 - val_acc: 0.5865

Epoch 00031: saving model t

Epoch 66/300
4903/4903 [==============================] - 0s 32us/step - loss: 1.0914 - acc: 0.6518 - val_loss: 1.1868 - val_acc: 0.6215

Epoch 00066: saving model to models/bert_text_Adam_lr0.0001_bz256.66.hdf5
Epoch 67/300
4903/4903 [==============================] - 0s 33us/step - loss: 1.0772 - acc: 0.6588 - val_loss: 1.1846 - val_acc: 0.6207

Epoch 00067: saving model to models/bert_text_Adam_lr0.0001_bz256.67.hdf5
Epoch 68/300
4903/4903 [==============================] - 0s 30us/step - loss: 1.0734 - acc: 0.6582 - val_loss: 1.1828 - val_acc: 0.6215

Epoch 00068: saving model to models/bert_text_Adam_lr0.0001_bz256.68.hdf5
Epoch 69/300
4903/4903 [==============================] - 0s 29us/step - loss: 1.0732 - acc: 0.6618 - val_loss: 1.1801 - val_acc: 0.6175

Epoch 00069: saving model to models/bert_text_Adam_lr0.0001_bz256.69.hdf5
Epoch 70/300
4903/4903 [==============================] - 0s 39us/step - loss: 1.0606 - acc: 0.6629 - val_loss: 1.1779 - val_acc: 0.6207

Epoch 00070: s


Epoch 00104: saving model to models/bert_text_Adam_lr0.0001_bz256.104.hdf5
Epoch 105/300
4903/4903 [==============================] - 0s 36us/step - loss: 0.9254 - acc: 0.7022 - val_loss: 1.1315 - val_acc: 0.6346

Epoch 00105: saving model to models/bert_text_Adam_lr0.0001_bz256.105.hdf5
Epoch 106/300
4903/4903 [==============================] - 0s 33us/step - loss: 0.9256 - acc: 0.6990 - val_loss: 1.1306 - val_acc: 0.6346

Epoch 00106: saving model to models/bert_text_Adam_lr0.0001_bz256.106.hdf5
Epoch 107/300
4903/4903 [==============================] - 0s 26us/step - loss: 0.9194 - acc: 0.7049 - val_loss: 1.1301 - val_acc: 0.6354

Epoch 00107: saving model to models/bert_text_Adam_lr0.0001_bz256.107.hdf5
Epoch 108/300
4903/4903 [==============================] - 0s 26us/step - loss: 0.9180 - acc: 0.7061 - val_loss: 1.1294 - val_acc: 0.6354

Epoch 00108: saving model to models/bert_text_Adam_lr0.0001_bz256.108.hdf5
Epoch 109/300
4903/4903 [==============================] - 0s 28us/s

Epoch 143/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.8131 - acc: 0.7434 - val_loss: 1.1084 - val_acc: 0.6370

Epoch 00143: saving model to models/bert_text_Adam_lr0.0001_bz256.143.hdf5
Epoch 144/300
4903/4903 [==============================] - 0s 34us/step - loss: 0.8178 - acc: 0.7389 - val_loss: 1.1096 - val_acc: 0.6387

Epoch 00144: saving model to models/bert_text_Adam_lr0.0001_bz256.144.hdf5
Epoch 145/300
4903/4903 [==============================] - 0s 34us/step - loss: 0.8093 - acc: 0.7434 - val_loss: 1.1087 - val_acc: 0.6362

Epoch 00145: saving model to models/bert_text_Adam_lr0.0001_bz256.145.hdf5
Epoch 146/300
4903/4903 [==============================] - 0s 28us/step - loss: 0.8050 - acc: 0.7428 - val_loss: 1.1089 - val_acc: 0.6378

Epoch 00146: saving model to models/bert_text_Adam_lr0.0001_bz256.146.hdf5
Epoch 147/300
4903/4903 [==============================] - 0s 32us/step - loss: 0.8150 - acc: 0.7361 - val_loss: 1.1092 - val_acc: 0.6403

Epoch

4903/4903 [==============================] - 0s 35us/step - loss: 0.7254 - acc: 0.7671 - val_loss: 1.1027 - val_acc: 0.6452

Epoch 00181: saving model to models/bert_text_Adam_lr0.0001_bz256.181.hdf5
Epoch 182/300
4903/4903 [==============================] - 0s 27us/step - loss: 0.7042 - acc: 0.7793 - val_loss: 1.1038 - val_acc: 0.6468

Epoch 00182: saving model to models/bert_text_Adam_lr0.0001_bz256.182.hdf5
Epoch 183/300
4903/4903 [==============================] - 0s 31us/step - loss: 0.7246 - acc: 0.7661 - val_loss: 1.1030 - val_acc: 0.6476

Epoch 00183: saving model to models/bert_text_Adam_lr0.0001_bz256.183.hdf5
Epoch 184/300
4903/4903 [==============================] - 0s 27us/step - loss: 0.7135 - acc: 0.7712 - val_loss: 1.1020 - val_acc: 0.6419

Epoch 00184: saving model to models/bert_text_Adam_lr0.0001_bz256.184.hdf5
Epoch 185/300
4903/4903 [==============================] - 0s 25us/step - loss: 0.7048 - acc: 0.7763 - val_loss: 1.1013 - val_acc: 0.6468

Epoch 00185: saving

4903/4903 [==============================] - 0s 33us/step - loss: 0.6375 - acc: 0.8022 - val_loss: 1.1049 - val_acc: 0.6476

Epoch 00219: saving model to models/bert_text_Adam_lr0.0001_bz256.219.hdf5
Epoch 220/300
4903/4903 [==============================] - 0s 25us/step - loss: 0.6421 - acc: 0.7940 - val_loss: 1.1064 - val_acc: 0.6452

Epoch 00220: saving model to models/bert_text_Adam_lr0.0001_bz256.220.hdf5
Epoch 221/300
4903/4903 [==============================] - 0s 35us/step - loss: 0.6381 - acc: 0.7999 - val_loss: 1.1066 - val_acc: 0.6436

Epoch 00221: saving model to models/bert_text_Adam_lr0.0001_bz256.221.hdf5
Epoch 222/300
4903/4903 [==============================] - 0s 27us/step - loss: 0.6420 - acc: 0.7962 - val_loss: 1.1056 - val_acc: 0.6460

Epoch 00222: saving model to models/bert_text_Adam_lr0.0001_bz256.222.hdf5
Epoch 223/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.6292 - acc: 0.7999 - val_loss: 1.1064 - val_acc: 0.6444

Epoch 00223: saving

4903/4903 [==============================] - 0s 32us/step - loss: 0.5684 - acc: 0.8207 - val_loss: 1.1135 - val_acc: 0.6444

Epoch 00257: saving model to models/bert_text_Adam_lr0.0001_bz256.257.hdf5
Epoch 258/300
4903/4903 [==============================] - 0s 28us/step - loss: 0.5731 - acc: 0.8250 - val_loss: 1.1145 - val_acc: 0.6444

Epoch 00258: saving model to models/bert_text_Adam_lr0.0001_bz256.258.hdf5
Epoch 259/300
4903/4903 [==============================] - 0s 37us/step - loss: 0.5749 - acc: 0.8226 - val_loss: 1.1120 - val_acc: 0.6436

Epoch 00259: saving model to models/bert_text_Adam_lr0.0001_bz256.259.hdf5
Epoch 260/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.5564 - acc: 0.8236 - val_loss: 1.1136 - val_acc: 0.6460

Epoch 00260: saving model to models/bert_text_Adam_lr0.0001_bz256.260.hdf5
Epoch 261/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.5608 - acc: 0.8275 - val_loss: 1.1137 - val_acc: 0.6419

Epoch 00261: saving

4903/4903 [==============================] - 0s 26us/step - loss: 0.5269 - acc: 0.8409 - val_loss: 1.1245 - val_acc: 0.6444

Epoch 00295: saving model to models/bert_text_Adam_lr0.0001_bz256.295.hdf5
Epoch 296/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.5108 - acc: 0.8421 - val_loss: 1.1264 - val_acc: 0.6452

Epoch 00296: saving model to models/bert_text_Adam_lr0.0001_bz256.296.hdf5
Epoch 297/300
4903/4903 [==============================] - 0s 26us/step - loss: 0.5125 - acc: 0.8344 - val_loss: 1.1267 - val_acc: 0.6460

Epoch 00297: saving model to models/bert_text_Adam_lr0.0001_bz256.297.hdf5
Epoch 298/300
4903/4903 [==============================] - 0s 29us/step - loss: 0.5157 - acc: 0.8368 - val_loss: 1.1259 - val_acc: 0.6468

Epoch 00298: saving model to models/bert_text_Adam_lr0.0001_bz256.298.hdf5
Epoch 299/300
4903/4903 [==============================] - 0s 30us/step - loss: 0.5112 - acc: 0.8440 - val_loss: 1.1268 - val_acc: 0.6444

Epoch 00299: saving

In [16]:
epoch = input("\n\nWhich epoch to load?\nAns: ")
epoch = int(epoch)
load_model_path = '%s.%02d.hdf5' % (model_path, epoch)
print('Loading model - ', load_model_path)
model = load_model(load_model_path, custom_objects={'loss': 'categorical_crossentropy'})
# print(model.summary())
predictions = model.predict(dev_articles_emb)
predictions = predictions.argmax(axis=1)

# writing predictions to file
task_TC_output_file = "BERT_model-output-TC.txt"
dev_article_ids, dev_span_starts, dev_span_ends, dev_labels = read_predictions_from_file(dev_template_labels_file)

with open(task_TC_output_file, "w") as fout:
    for article_id, prediction, span_start, span_end in zip(dev_article_ids, predictions, dev_span_starts,
                                                            dev_span_ends):
        fout.write("%s\t%s\t%s\t%s\n" % (article_id, index2label[prediction], span_start, span_end))
print("Predictions written to file " + task_TC_output_file)



Which epoch to load?
Ans: 189
Loading model -  models/bert_text_Adam_lr0.0001_bz256.189.hdf5
Predictions written to file BERT_model-output-TC.txt
